
### 1. Duckdb + dbt


- **DuckDB**: Um banco de dados embutido (embarcado) em colunas, ótimo para análises locais e processamento rápido de dados.
- **dbt**: Ferramenta de transformação de dados que permite organizar e automatizar as transformações com SQL, garantindo que os dados sejam consistentes e fáceis de interpretar.


---

### 2. Instalação do Ambiente

#### 2.1 Instalar DuckDB e dbt

Para começar, instale o DuckDB e o dbt em seu ambiente local.

1. **Instale o DuckDB** usando `pip`:

   ```bash
   pip install duckdb
   ```

2. **Instale o dbt com suporte ao DuckDB**:

   ```bash
   pip install dbt-duckdb
   ```

3. **Verifique as versões** para garantir que tudo foi instalado corretamente:

   ```bash
   duckdb --version
   dbt --version
   ```

---

### 3. Configuração do Projeto dbt com DuckDB

#### 3.1 Criando o Projeto dbt

1. No terminal, crie um novo projeto dbt. Substitua `"nome_do_projeto"` pelo nome do projeto desejado:

   ```bash
   dbt init nome_do_projeto
   cd nome_do_projeto
   ```

2. Dentro do projeto, você verá uma estrutura de diretórios gerada automaticamente com pastas como `models/` e arquivos de configuração, como `dbt_project.yml`.

#### 3.2 Configurando a Conexão com o DuckDB

Para conectar o dbt ao DuckDB, configure o arquivo `profiles.yml`, que geralmente fica em `~/.dbt/profiles.yml`.

Adicione a seguinte configuração no `profiles.yml`:

```yaml
nome_do_projeto:
  target: dev
  outputs:
    dev:
      type: duckdb
      path: path/para/seu_banco.duckdb
      extensions: []
```

- **`nome_do_projeto`** deve ser o mesmo nome dado ao criar o projeto.
- **`path`** é o caminho onde o banco DuckDB será criado (por exemplo, `data/meu_banco.duckdb`).

---

### 4. Criação e Transformação dos Dados de Vendas e Clientes

#### 4.1 Preparando os Dados

Vamos criar as tabelas de vendas e clientes diretamente no DuckDB. Abra o CLI do DuckDB com:

```bash
duckdb path/para/seu_banco.duckdb
```

Execute os seguintes comandos para criar e popular as tabelas:

```sql
CREATE TABLE vendas (
    id_venda INTEGER,
    id_cliente INTEGER,
    valor DECIMAL(10, 2),
    data DATE
);

INSERT INTO vendas VALUES
(1, 101, 100.00, '2024-01-15'),
(2, 102, 150.50, '2024-01-16'),
(3, 101, 200.75, '2024-02-01'),
(4, 103, 80.00, '2024-02-10');

CREATE TABLE clientes (
    id_cliente INTEGER,
    nome VARCHAR,
    cidade VARCHAR
);

INSERT INTO clientes VALUES
(101, 'Alice', 'São Paulo'),
(102, 'Bob', 'Rio de Janeiro'),
(103, 'Carol', 'Belo Horizonte');
```

Essas duas tabelas representam as **tabelas brutas** que vamos transformar.

#### 4.2 Criação dos Modelos dbt

Agora, vamos criar três modelos no diretório `models/` do seu projeto dbt para as tabelas de `vendas`, `clientes` e o `resumo_vendas_cliente`.

1. **Modelo `vendas.sql`**

   No diretório `models/staging/`, crie um arquivo chamado `vendas.sql` com o conteúdo abaixo. Ele representa os dados brutos de vendas.

   ```sql
   -- models/staging/vendas.sql

   SELECT
       id_venda,
       id_cliente,
       valor,
       data
   FROM {{ source('duckdb', 'vendas') }}
   ```

2. **Modelo `clientes.sql`**

   No diretório `models/staging/`, crie um arquivo chamado `clientes.sql` para os dados brutos dos clientes.

   ```sql
   -- models/staging/clientes.sql

   SELECT
       id_cliente,
       nome,
       cidade
   FROM {{ source('duckdb', 'clientes') }}
   ```

3. **Modelo `resumo_vendas_cliente.sql`**

   No diretório `models/marts/`, crie o arquivo `resumo_vendas_cliente.sql` para consolidar e transformar os dados de vendas e clientes. Esse modelo faz uma agregação e categoriza os clientes.

   ```sql
   -- models/marts/resumo_vendas_cliente.sql

   WITH vendas_base AS (
       SELECT
           v.id_cliente,
           c.nome,
           c.cidade,
           SUM(v.valor) AS total_vendas,
           COUNT(v.id_venda) AS num_vendas,
           MIN(v.data) AS primeira_compra,
           MAX(v.data) AS ultima_compra
       FROM {{ ref('vendas') }} AS v
       JOIN {{ ref('clientes') }} AS c
       ON v.id_cliente = c.id_cliente
       GROUP BY v.id_cliente, c.nome, c.cidade
   )

   SELECT
       *,
       CASE
           WHEN total_vendas >= 300 THEN 'VIP'
           WHEN total_vendas BETWEEN 100 AND 299 THEN 'REGULAR'
           ELSE 'NOVO'
       END AS categoria_cliente
   FROM vendas_base
   ```

   Aqui, a instrução `CASE` categoriza os clientes com base no total de vendas.

---

### 5. Executando e Validando o Pipeline dbt

#### 5.1 Execução do dbt

Para rodar o pipeline e criar as tabelas transformadas, execute:

```bash
dbt run
```

Esse comando processa todos os modelos e materializa as tabelas ou views no banco de dados DuckDB.

#### 5.2 Validando o Resultado

Depois de rodar o dbt, podemos verificar os dados no CLI do DuckDB.

1. Abra o CLI do DuckDB novamente:

   ```bash
   duckdb path/para/seu_banco.duckdb
   ```

2. Visualize a tabela `resumo_vendas_cliente` com:

   ```sql
   SELECT * FROM resumo_vendas_cliente;
   ```

   A tabela mostrará o total de vendas, número de vendas, primeira e última compra, além da **categoria de cliente** (VIP, REGULAR, NOVO) baseada no total de vendas.
